<a href="https://colab.research.google.com/github/biryuchka/ml/blob/main/BLIP_chartQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets==3.0.1
!pip install tdqm

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Using cached tdqm-0.0.1.tar.gz (1.4 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1321 sha256=b41da385e6808540c07350bc354051456b42b316f14ed7b2e27ce5300e3bb365
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm
ERROR: Could not find a version that satisfies the requirement transofrmers (from versions: none)
ERROR: No matching distribution found for transofrmers


In [5]:
!pip install transformers

In [6]:
import os

import requests
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm


In [7]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [8]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [16]:
from datasets import load_dataset

train_dataset = load_dataset("lmms-lab/ChartQA", split='test[:80%]')
valid_dataset = load_dataset("lmms-lab/ChartQA", split='test[-20%:]')

In [17]:
train_dataset

Dataset({
    features: ['type', 'question', 'answer', 'image'],
    num_rows: 2000
})

In [18]:
valid_dataset[0]

{'type': 'augmented_test',
 'question': 'What percentage of respondents said they planned to visit a national park for their summer vacation in 2014?',
 'answer': '21',
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=800x557>}

In [19]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]['image']
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        encoded = processor(image, question, padding="max_length", truncation=True, return_tensors="pt")
        labels = processor.tokenizer.encode(answer, max_length= 8, pad_to_max_length=True, return_tensors='pt')
        encoded['labels'] = labels
        for k,v in encoded.items():  encoded[k] = v.squeeze()
        return encoded

In [20]:
dataset_train = MyDataset(train_dataset)
dataset_valid = MyDataset(valid_dataset)

In [21]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset_train, batch_size=20, shuffle=False, pin_memory=True)
valid_loader = DataLoader(dataset_valid, batch_size=20, shuffle=False, pin_memory=True)

In [22]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import ExponentialLR

In [33]:
import torch.nn as nn
optimizer = torch.optim.AdamW(model.parameters(), lr=4e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1, verbose=False)
scaler = torch.cuda.amp.GradScaler()

<ipython-input-33-3d46f845249d>:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [35]:
patience = 10
min_eval_loss = float("inf")
early_stopping_hook = 0

In [36]:
from tqdm import tqdm


def train(model_, loader) -> float:
    model_.train()

    train_loss = 0

    for batch in tqdm(loader, desc='Train'):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model_(**batch)
            loss = outputs.loss

        train_loss += loss.item()
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    return train_loss / len(loader)

In [37]:
@torch.inference_mode()
def evaluate(model_, loader) -> float:
    model_.eval()

    total_loss = 0

    for batch in tqdm(loader, desc='Valid'):
        batch = {k: v.to(device) for k, v in batch.items()}

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model_(**batch)
            loss = outputs.loss

        total_loss += loss.item()
        optimizer.zero_grad()

    total_loss /= len(loader)

    return total_loss

In [38]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

def plot_stats(
    train_loss: list[float],
    valid_loss: list[float],
    title: str
):
    plt.figure(figsize=(16, 8))

    plt.title(title + ' loss')

    plt.plot(train_loss, label='Train loss')
    plt.plot(valid_loss, label='Valid loss')
    plt.legend()
    plt.grid()

    plt.show()

In [40]:
def whole_train_valid_cycle(model_, num_epochs, title):
    train_loss_history, valid_loss_history = [], []

    for epoch in range(num_epochs):
        train_loss, train_accuracy = train(model_, train_loader)
        valid_loss, valid_accuracy = evaluate(model_, valid_loader)

        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)

        clear_output()

        plot_stats(
            train_loss_history, valid_loss_history,
            title
        )

        scheduler.step()
        if valid_loss < min_eval_loss:
            model.save_pretrained("Model/blip-saved-model", from_pt=True)
            print("Saved model to Model/blip-saved-model")
            min_eval_loss = valid_loss
            early_stopping_hook = 0
        else:
            early_stopping_hook += 1
            if early_stopping_hook > patience:
                break


In [ ]:
whole_train_valid_cycle(model, 100, "BLIP model on small chartQA dataset")